In [1]:
import pandas as pd

In [2]:


#RBH results
df_rbh = pd.read_csv('../../../results/reciprocal_best_hit_TSV/rbh_all_in_one_file.tsv', sep='\t', index_col=0)

#model organims annotation data
df_MO_annotation = pd.read_csv('../../../results/Gene_annotation_info_from_uniprot_model_spp.tsv', sep='\t', low_memory=False, index_col=0)

#kinetoplastea annotation data
#one structure per row
df_kineto_annotation = pd.read_csv('../../protein_data_bases/annotation_info/kinetoplastea_taxid5653_annotation_info_from_uniprot.tsv', sep='\t', low_memory=False)


#estructure to genID to OG

df_OG_genID_uniprot  = pd.read_csv('../../report/ortholog_groups_x_sequence_clustering_x_UNIPROT.tsv', sep='\t')

#hypothetical OG data
df_hypothetical_OG = pd.read_csv('../../report/Ortholog_Group_Full_of_hypotetical_boolean.tsv', sep='\t')

In [17]:
df_hypothetical_OG.only_hypothetical_OG.value_counts()

False    10081
True      2920
Name: only_hypothetical_OG, dtype: int64

In [3]:
df_rbh_k = df_rbh.merge(df_kineto_annotation, left_on='query_uniprot_accession', right_on='Entry', how='left')

In [4]:
df_rbh_km = df_rbh_k.merge(df_MO_annotation, left_on='target_uniprot_accession', right_on='Entry', how='left', suffixes=['_kineto', '_model'])

In [5]:
df_rbh_km = df_rbh_km[~df_rbh_km.spp.isin(['LEIIN', 'TRYCC', 'TRYB2'])]

In [6]:
df_rbh_km.shape

(168062, 72)

In [7]:
df_rbh_km = df_rbh_km.merge(df_OG_genID_uniprot, left_on='query_uniprot_accession', right_on='uniprot', how='left')

In [8]:
df_rbh_km = df_rbh_km.merge(df_hypothetical_OG, left_on='Ortholog_Group', right_on='cluster_representer_or_OG')

# selection best RBH 

basado en la notebook 4 nos quedamos con el best RBH y esa es la anotacion

In [18]:
df_rbh_km_BH = df_rbh_km.sort_values('evalue').drop_duplicates(subset=['query'], keep='first')

In [19]:
df_rbh_km_BH.only_hypothetical_OG.value_counts()

False    8214
True     2294
Name: only_hypothetical_OG, dtype: int64

In [52]:
df_rbh_km_BH_filtered = df_rbh_km_BH[(df_rbh_km_BH['fident'] > 0) & (df_rbh_km_BH['evalue'] < 1e-2)]#OGsize
df_rbh_km_BH_filtered.only_hypothetical_OG.value_counts()

False    7128
True     1567
Name: only_hypothetical_OG, dtype: int64

In [50]:
df_rbh_km_BH_filtered[df_rbh_km_BH_filtered.only_hypothetical_OG == True].sort_values('fident')#.head(50)

,query,target,alnlen,fident,evalue,qstart,qend,qlen,tstart,tend,...,InterPro_model,Pfam_model,Ortholog_Group,Gene ID,uniprot,pLDDT_mean,cluster_representer_or_OG,OGsize,OGhypotetical_proteins,only_hypothetical_OG
148893,AF-A0A0N0DSN2-F1-model_v4.pdb,AF-Q54T41-F1-model_v4.cif.gz,193,0.058,6.974000e-13,48,542,543,0,399,...,IPR000315;,PF00643;,GH5_06787,LpyrH10_21_0790,A0A0N0DSN2,67.0,GH5_06787,35,35.0,True
167381,AF-A4HS87-F1-model_v4.pdb,AF-Q8ILQ6-F1-model_v4.cif.gz,92,0.061,2.929000e-10,0,1835,1872,0,2177,...,NaN,NaN,CFAC1_030018700,LINF_040014000,A4HS87,33.0,CFAC1_030018700,24,24.0,True
62085,AF-A4HZE8-F1-model_v4.pdb,AF-Q8IJQ2-F1-model_v4.cif.gz,81,0.065,6.986000e-11,0,1462,1465,676,2218,...,IPR001870;IPR043136;IPR013320;IPR013144;IPR003...,PF00622;,LbrM.21.1650,LINF_210023700,A4HZE8,41.0,LbrM.21.1650,34,34.0,True
167294,AF-A0A0M9FQZ7-F1-model_v4.pdb,AF-C6S3B8-F1-model_v4.cif.gz,54,0.068,2.074000e-06,27,814,815,0,800,...,IPR024771;,PF12752;,CFAC1_160019600,LpyrH10_31_1110,A0A0M9FQZ7,44.0,CFAC1_160019600,31,31.0,True
19784,AF-A0A3P3YZN3-F1-model_v4.pdb,AF-Q04368-F1-model_v4.cif.gz,177,0.071,1.400000e-08,2,427,474,1,293,...,NaN,NaN,LtaPh_1003200,LbrM.10.0360,A0A3P3YZN3,61.0,LtaPh_1003200,35,35.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14392,AF-G0TRQ1-F1-model_v4.pdb,AF-O13804-F1-model_v4.cif.gz,223,0.256,4.126000e-08,0,119,125,0,119,...,IPR009018;IPR039914;IPR039432;,PF05486;,TM35_000391730,TvY486_0200420,G0TRQ1,77.0,TM35_000391730,21,21.0,True
26046,AF-A0A640KUP6-F1-model_v4.pdb,AF-Q9V8I2-F1-model_v4.pdb.gz,323,0.260,6.343000e-10,0,137,138,0,124,...,IPR037587;IPR004942;,PF03259;,BSAL_38745,LtaPh_3545900,A0A640KUP6,93.0,BSAL_38745,57,57.0,True
53076,AF-A4IDC0-F1-model_v4.pdb,AF-A0A0R0ENU4-F1-model_v4.pdb.gz,259,0.272,7.560000e-08,0,167,174,0,151,...,NaN,NaN,JKF63_00140,LINF_360021300,A4IDC0,77.0,JKF63_00140,29,29.0,True
119505,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-A0A0D2GX90-F1-model_v4.pdb.gz,294,0.280,7.429000e-10,5,108,109,1,101,...,IPR013097;IPR011008;,PF07876;,PCON_0085960,BSAL_46720,A0A0S4JUQ9,94.0,PCON_0085960,15,15.0,True


In [51]:



df_rbh[df_rbh.query_uniprot_accession == 'A0A0S4JUQ9' ].sort_values('evalue')

,query,target,alnlen,fident,evalue,qstart,qend,qlen,tstart,tend,tlen,aligmentinfo,proteome,spp,query_uniprot_accession,target_uniprot_accession,COV_query,COV_target
55,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-A0A0D2GX90-F1-model_v4.pdb.gz,294,0.280,7.429000e-10,5,108,109,1,101,102,13M1I11M3D16M4I46M1I12M,UP000053029,9EURO2,A0A0S4JUQ9,A0A0D2GX90,0.94,0.98
56,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-A0A0D2GX90-F1-model_v4.cif.gz,294,0.280,7.429000e-10,5,108,109,1,101,102,13M1I11M3D16M4I46M1I12M,UP000053029,9EURO2,A0A0S4JUQ9,A0A0D2GX90,0.94,0.98
3337,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-A0A1C1CSM9-F1-model_v4.cif.gz,289,0.271,1.136000e-09,5,108,109,1,101,102,13M1I11M3D16M4I46M1I12M,UP000094526,9EURO1,A0A0S4JUQ9,A0A1C1CSM9,0.94,0.98
3336,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-A0A1C1CSM9-F1-model_v4.pdb.gz,289,0.271,1.136000e-09,5,108,109,1,101,102,13M1I11M3D16M4I46M1I12M,UP000094526,9EURO1,A0A0S4JUQ9,A0A1C1CSM9,0.94,0.98
1263,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-C1HD18-F1-model_v4.cif.gz,292,0.266,1.775000e-09,5,106,109,1,99,100,13M1I4M3D23M4I48M1I8M,UP000002059,PARBA,A0A0S4JUQ9,C1HD18,0.93,0.98
1262,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-C1HD18-F1-model_v4.pdb.gz,292,0.266,1.775000e-09,5,106,109,1,99,100,13M1I4M3D23M4I48M1I8M,UP000002059,PARBA,A0A0S4JUQ9,C1HD18,0.93,0.98
2411,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-C6SXJ3-F1-model_v4.cif.gz,287,0.198,2.683000e-09,3,108,109,0,99,105,15M1I29M1I5M3I39M1I12M,UP000008827,SOYBN,A0A0S4JUQ9,C6SXJ3,0.96,0.94
526,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-Q8LQD2-F1-model_v4.pdb.gz,257,0.223,3.454000e-09,1,108,109,4,108,113,4M1D14M1I14M3D12M1I8M4I37M1I12M,UP000059680,ORYSJ,A0A0S4JUQ9,Q8LQD2,0.98,0.92
527,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-Q8LQD2-F1-model_v4.cif.gz,257,0.223,3.454000e-09,1,108,109,4,108,113,4M1D14M1I14M3D12M1I8M4I37M1I12M,UP000059680,ORYSJ,A0A0S4JUQ9,Q8LQD2,0.98,0.92
3591,AF-A0A0S4JUQ9-F1-model_v4.pdb,AF-C0NKK7-F1-model_v4.cif.gz,273,0.257,5.324000e-09,5,106,109,1,99,100,13M1I11M3D16M4I48M1I8M,UP000001631,AJECG,A0A0S4JUQ9,C0NKK7,0.93,0.98


In [14]:
#df_rbh_km_BH_filtered.iloc[:,:12].to_csv('../../tmp/')

In [46]:
!grep 'Baya_067_0220' ../../../config/mandatory_files/Ortholog_group_to_geneID.tsv

Baya_067_0220	Baya_067_0220
Baya_067_0220	BSAL_86340
Baya_067_0220	LpyrH10_23_1270
Baya_067_0220	Lsey_0135_0150
Baya_067_0220	CFAC1_010006900
Baya_067_0220	LMARLEM2494_120006400
Baya_067_0220	JIQ42_08314
Baya_067_0220	LENLEM3045_120006500
Baya_067_0220	LSCM4_07603
Baya_067_0220	GH5_08296
Baya_067_0220	LSCM1_07822
Baya_067_0220	CUR178_08239


In [39]:
df_OG_genID_uniprot[df_OG_genID_uniprot.Ortholog_Group == 'LAMA_000335000']

,Ortholog_Group,Gene ID,uniprot,pLDDT_mean
5128,LAMA_000335000,TcCLB.507681.130,Q4DW40,78.0


In [45]:
df_OG_genID_uniprot[df_OG_genID_uniprot.uniprot == 'A0A0S4J829']

,Ortholog_Group,Gene ID,uniprot,pLDDT_mean
9804,Baya_067_0220,BSAL_86340,A0A0S4J829,59.0
